In [1]:
import anndata as ad
import networkx as nx
import scanpy as sc

from itertools import chain
from matplotlib import rcParams
import pandas as pd
import scglue
import os
print(os.getcwd())
path='/home/ubuntu0/scBasset/'
#

rna = ad.read_h5ad(path+"Chen-2019-RNA.h5ad")
atac = ad.read_h5ad(path+"Chen-2019-ATAC.h5ad")
rna.layers["counts"] = rna.X.copy()

#select high expressive genes as in the tutorial the number 2000
sc.pp.highly_variable_genes(rna, n_top_genes=2000, flavor="seurat_v3")

sc.pp.normalize_total(rna)
sc.pp.log1p(rna)
sc.pp.scale(rna)
sc.tl.pca(rna, n_comps=100, svd_solver="auto")

sc.pp.neighbors(rna, metric="cosine")
sc.tl.umap(rna)

scglue.data.lsi(atac, n_components=100, n_iter=15)#embedding is computed before the subsetting 
# sc.pp.subsample(rna, n_obs=2000, random_state=42)

scglue.data.get_gene_annotation(
    rna, gtf="/home/ubuntu0/GLUE/evaluation/workflow/scripts/gencode.vM25.chr_patch_hapl_scaff.annotation.gtf.gz",
    gtf_by="gene_name"
)

print(rna.var.loc[:,["chrom","chromStart","chromEnd"]].head())
# sc.pp.filter_cells(atac, min_genes=0)
# sc.pp.filter_genes(atac, min_cells=0)

split = atac.var_names.str.split(r"[:-]")
atac.var["chrom"] = split.map(lambda x: x[0])
atac.var["chromStart"] = split.map(lambda x: x[1]).astype(int)
atac.var["chromEnd"] = split.map(lambda x: x[2]).astype(int)
print(atac.var.head())

guidance=scglue.genomics.rna_anchored_guidance_graph(rna, atac)

atac=atac[pd.Series(atac.obs_names).sample(2000),pd.Series(atac.var_names).sample(30000)].copy()
sc.pp.subsample(rna, n_obs=2000, random_state=42)



/home/ubuntu0/miniconda3/envs/glue2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/ubuntu0/GLUE/paired_data


/home/ubuntu0/miniconda3/envs/glue2/lib/python3.8/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


               chrom  chromStart   chromEnd
genes                                      
0610005C13Rik   chr7    45567793   45575327
0610009B22Rik  chr11    51685385   51688874
0610009E02Rik   chr2    26445695   26459390
0610009L18Rik  chr11   120348677  120351190
0610010F05Rik  chr11    23564960   23633639
                     chrom  chromStart  chromEnd
peaks                                           
chr1:3005833-3005982  chr1     3005833   3005982
chr1:3094772-3095489  chr1     3094772   3095489
chr1:3119556-3120739  chr1     3119556   3120739
chr1:3121334-3121696  chr1     3121334   3121696
chr1:3134637-3135032  chr1     3134637   3135032


window_graph: 100%|████████████████████████████████████████████████████████████████████████| 28930/28930 [00:02<00:00, 13759.75it/s]


In [2]:
rna,atac

(AnnData object with n_obs × n_vars = 2000 × 28930
     obs: 'domain', 'cell_type'
     var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std', 'chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts', 'gene_id', 'gene_type', 'mgi_id', 'havana_gene', 'tag'
     uns: 'hvg', 'log1p', 'pca', 'neighbors', 'umap'
     obsm: 'X_pca', 'X_umap'
     varm: 'PCs'
     layers: 'counts'
     obsp: 'distances', 'connectivities',
 AnnData object with n_obs × n_vars = 2000 × 30000
     obs: 'domain', 'cell_type'
     var: 'chrom', 'chromStart', 'chromEnd', 'highly_variable'
     obsm: 'X_lsi')

In [4]:
rna.write_h5ad('rna1129.h5ad')
atac.write_h5ad('atac1129.h5ad')
nx.write_graphml(guidance,'full_guidance.graphml.gz')